In [3]:
import time
import pycurl
import urllib
import json

In [13]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/prices'

class SwissQuoteStream:
    def __init__(self):
        self.conn = None
        self.buffer = ''
        self.setup_connection()
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL)
        self.conn.setopt(pycurl.WRITEFUNCTION, self.handle_price)
    
    def start(self):
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error: %s', self.conn.errstr())
                print 'Waiting %s seconds before trying again' % backoff_network_error
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
        
    def handle_price(self, data):
        self.buffer += data
        if data.endswith('\r\n') and self.buffer.strip():
            # complete message received
            message = json.loads(self.buffer)
            self.buffer = ''
            msg = ''
            print(msg)
            
if __name__ == '__main__':
    stream = SwissQuoteStream()
    stream.setup_connection()
stream.start()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(print 'Waiting %s seconds before trying again' % backoff_network_error)? (<ipython-input-13-518d1658de08>, line 27)